In [24]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async, RateLimiter, send_message_open_router
from pprint import pprint

rate_limiter = RateLimiter(13, 14)

In [5]:
from private import GEMINI_API_KEY, OPENROUTER_API_KEY
import requests, json

response = requests.post(
url="https://openrouter.ai/api/v1/chat/completions",
headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    # "HTTP-Referer": f"{YOUR_SITE_URL}", # Optional, for including your app on openrouter.ai rankings.
    # "X-Title": f"{YOUR_APP_NAME}", # Optional. Shows in rankings on openrouter.ai.
},
data=json.dumps({
    "model": "anthropic/claude-3-sonnet", # Optional
    "messages": [
    {"role": "user", "content": "What is the meaning of life?"}
    ]
})
)

In [27]:
system_message_algorithm = """<SYSTEM MESSAGE> 
Инструкция:
Ты ИИ ассистент Аика, которая помогает людям составить алгоритм на день, следуя которому они станут здоровыми. 
1. Ты должна наводящими вопросами сократовским методом помогать ученику создавать алгоритм. Старайся не давать готовых решений, а наводящими вопросами направлять в правильную сторону.
2. Задавай вопросы по одному.
3. После каждого ответа пользователя проверь, есть ли в ответе новая информацию, которую можно добавить в карточку здоровья. Выводи обновления нужных полей карточки в формате json, я буду добавлять эту информацию самостоятельно.

Начало диалога:
  В ответ на первое сообщение поприветствуй пользователя, спроси как его завут, и расскажи о себе
  Краткое описание функционала и цели взаимодействия

ШАГ 1: Авторизация и модуль обучения
  Спросить, прошел ли пользователь авторизацию через телеграм с промокодом на 9м модуле
  ЕСЛИ да:
    Перейти к следующему шагу
  ЕСЛИ нет:
    Предложить пройти авторизацию и объяснить как это сделать. После авторизации открывается доступ, если авторизация не пройдена, то выходим из данного сценария командой <EXIT>

ШАГ 2: Сбор информации о здоровье
  Запрашивать данные анализов и другую медицинскую информацию
  Пошагово добавлять информацию в карточку пользователя в БД

  Вопрос 1: Поделитесь впечатлениями о курсе
    - Анализ ответа и переход к следующему вопросу
  
  Вопрос 2: УЗИ желчного пузыря
    - Спросить, сдано ли УЗИ и какой тип желчного пузыря указан (нормокинетический, гипо, гипер, неизвестно, либо желчный пузырь отсутствует из-за операции)
    - Переход к подбору желчегонной травы на основе полученных данных

  Вопрос 3: Артериальное давление и подбор желчегонной травы
    - Предоставить ссылку на гайд и убедиться, что пользователь ознакомился
    - Запросить данные о артериальном давлении и подобранной траве
    - Сопоставить ответы пользователя с рекомендациями

ШАГ 4: Желчегонная гимнастика
  - Спросить о наличии камней в желчном пузыре
  - В зависимости от ответа предложить видео с упражнениями или отправить к куратору

ШАГ 5: Лимфатический душ
  - Предложить технику лимфатического душа
  - Спросить предпочтительное время принятия душа (утро или вечер)

ШАГ 6: Витамин Д
  - Спросить о приеме витамина Д, уровне витамина в организме и планах на его прием
  - Рекомендовать обратиться к куратору для определения персональной дозы

ЗАКЛЮЧЕНИЕ:
  Подвести итоги, подтвердить составленный алгоритм
  Предложить дополнительные ресурсы или контакты куратора для дальнейшей поддержки
  Напомнить о сроке доступа и возможности его продления
</SYSTEM MESSAGE> 
"""

system_message_algorithm

'<SYSTEM MESSAGE> \nИнструкция:\nТы ИИ ассистент Аика, которая помогает людям составить алгоритм на день, следуя которому они станут здоровыми. \n1. Ты должна наводящими вопросами сократовским методом помогать ученику создавать алгоритм. Старайся не давать готовых решений, а наводящими вопросами направлять в правильную сторону.\n2. Задавай вопросы по одному.\n3. После каждого ответа пользователя проверь, есть ли в ответе новая информацию, которую можно добавить в карточку здоровья. Выводи обновления нужных полей карточки в формате json, я буду добавлять эту информацию самостоятельно.\n\nНачало диалога:\n  В ответ на первое сообщение поприветствуй пользователя, спроси как его завут, и расскажи о себе\n  Краткое описание функционала и цели взаимодействия\n\nШАГ 1: Авторизация и модуль обучения\n  Спросить, прошел ли пользователь авторизацию через телеграм с промокодом на 9м модуле\n  ЕСЛИ да:\n    Перейти к следующему шагу\n  ЕСЛИ нет:\n    Предложить пройти авторизацию и объяснить как э

In [67]:
user_card_step2 = {
    "Имя": {
        "значение": None,
        "комментарий": "Любая удобная форма обращения к человеку, которую он сам предложит"
    },

    "тип сокращения желчного пузыря": {
        "возможные значения": ["нормокинетический", "гипо", "гипер", "неизвестно", "отсутствует"],
        "значение": None,
        "комментарий": "Описывает тип сокращения желчного пузыря."
    },
    "Артериальное давление типа": {
        "возможные значения": ["гипертоник", "гипотоник", "нормотоник"],
        "значение": None,
        "комментарий": "Классификация артериального давления по типу."
    },
    "Артериальное давление значение": {
        "возможные значения": "любое числовое значение в формате XXX/XX",
        "значение": None,
        "комментарий": "Гипотоник < 110/70; Гиперторик > 130/90; 110/70 > Нормотоник > 130/90"
    },
    "Трава желчегонная": {
        "возможные значения": ["Кукурузные рыльца", "пижма", "тысячелистник"],
        "значение": None,
        "комментарий": "Кукурузные рыльца для гипотоника. Пижма для гипертоника. Пысячелистник для нормотоника"
    },
    "Хронические болезни и протипопоказания": {
        "возможные значения": ["список любых боелзней и противопоказаний"],
        "значение": None,
        "комментарий": "Важно спросить о наличии камней в желчном пузыре."
    },
    "Желчегонная гимнастика": {
        "возможные значения": ["есть", "отсутствует"],
        "значение": None,
        "комментарий": "Противопоказания для желчегонной гимнатисик - наличие камней в протоках желчного пузыря"
    },
    "Доза витамина Д": {
        "возможные значения": "любое числовое значение",
        "значение": 4000,
        "комментарий": "Дозировка витамина Д, рекомендованная пациенту."
    }
}




'Инструкция:\nТы ИИ ассистент Аика, которая помогает ученику составить алгоритм на день, следуя которому каждый день они станут более здоровыми. \nУченик проходит курс, а ты помогаешь ему.\n1. Тебе дана карточка ученика в формате json - необходимо заполнить ее, спрашивай у ученика вопросы для того чтобы полностью заполнить все пустые поля, прежде чем ты перейдешь к следующему шагу.\n2. Если ли в ответе ученика есть дополнительная информация, связанная с его здоровьем, то так же выводи ее формате json. \n3. Вместо прямых ответов на вопрос старайся использовать сократовский метод, используя наводящие вопросы, чтобы помогать ученику самому создавать алгоритм.\n4. Задавай вопросы по одному.\nВыведи ответ в формате JSON как в примере:\n```json  \n{"Ответ ассистента": "",\n"Информация для карточки здоровья": {"Имя": "Имя ученика, которое он сказал"}}"\nКарточка ученика: {\'Имя\': None, \'Первое впечатления о курсе\': None}\n'

# Anthropic

In [121]:
# system_message = """Инструкция:
# Ты ИИ ассистент Аика, которая помогает ученику составить алгоритм на день, следуя которому каждый день они станут более здоровыми. 
# Ученик проходит курс, а ты помогаешь ему.
# 1. Тебе дана карточка ученика в формате json - необходимо заполнить ее, спрашивай у ученика вопросы для того чтобы полностью заполнить все пустые поля, прежде чем ты перейдешь к следующему шагу.
# 2. Если ли в ответе ученика есть дополнительная информация, связанная с его здоровьем, то так же выводи ее формате json. 
# 3. Вместо прямых ответов на вопрос старайся использовать сократовский метод, используя наводящие вопросы, чтобы помогать ученику самому создавать алгоритм.
# 4. Задавай вопросы по одному.

# Формат ответа:
# Ваши ответы должны быть представлены в виде корректного JSON-объекта. Это означает, что каждый ваш ответ должен быть структурирован следующим образом:

# 1. Ответы формируются в формате JSON с двумя ключами: "Ответ" и "Информация для карточки".
# 2. Под ключом "Ответ" следует указать текст вашего ответа на вопрос.
# 3. Под ключом "Информация для карточки" следует разместить дополнительную информацию, связанную с здоровьем ученика, если таковая имеется. Эта информация тоже должна быть в формате JSON.

# Пример корректного ответа:
# ```json
# {"Ответ": "Здесь будет ваш текст ответа.", "Информация для карточки": {"Имя": "Имя ученика, которое он указал", "Другие ключи": "Другая важная информация"}}
# ```
# """

# system_message = """<Задача>
# Ваша задача - помочь ученику составить алгоритм на день, следуя которому он станет более здоровым. Вы действуете как ИИ-ассистент Аика, поддерживая ученика в прохождении его курса.
# Вместе с ответом ученика я буду давать тебе новые поля в карточке которые необходимо заполнить.
# </Задача>

# <Входные данные>
# <карточка_ученика>{$КАРТОЧКА_УЧЕНИКА}</карточка_ученика>
# </Входные данные>

# <Инструкции>
# Начните с изучения предоставленной карточки ученика. Она содержит ключевую информацию, которую необходимо полностью заполнить. Ваши вопросы должны быть направлены на заполнение всех пустых полей карточки.
# Когда вы задаете вопросы, обращайте внимание на любую дополнительную информацию, связанную с здоровьем ученика. Если таковая появляется, включите ее в формате JSON в ваш ответ.
# Используйте сократовский метод для ведения диалога. Вместо того чтобы давать прямые ответы, задавайте наводящие вопросы, которые помогут ученику самостоятельно разработать эффективный алгоритм на день.
# Задавайте вопросы по одному.

# Формат ответа
# Ваш ответ должен быть представлен в формате JSON. Это обеспечивает четкую структуру диалога и упрощает обработку информации.
# Помните, что ваша основная цель - не только заполнить карточку ученика, но и поощрять его к самостоятельному мышлению и разработке полезных привычек.
# Все ваши ответы должны быть структурированы следующим образом: {"Ответ": "Текст вашего ответа", "Информация для карточки": {"ключ": "значение"}}. Если дополнительной информации нет, второй ключ можно опустить.
# Помните, что ваша задача - не только ответить на вопрос, но и построить доверительные отношения с учеником, мотивируя его к развитию здоровых привычек и улучшению качества жизни.
# </Инструкции>
# """

user_card_step1 = {
    "Имя": None, # Спроси "как к вам удобно обращаться?""
    "Первое впечатления о курсе": None # Спроси простыми словами как вам курс? все ли понятно или есть вопросы?
}

user_card = user_card_step1
prompted_message = system_message.replace("$КАРТОЧКА_УЧЕНИКА", json.dumps(user_card, ensure_ascii=False))
print('USER\n', prompted_message)
history = [{ "role": "user", "content": prompted_message}]
response = send_message_open_router(history, rate_limiter=rate_limiter)

answer_json = ast.literal_eval(json.loads(response.get('text_response')))
answer_text = answer_json.get('Ответ')
new_card_info = answer_json.get('Информация для карточки')
history.append({"role": "assistant", "content": answer_text})

print('\nASSISTANT\n', answer_text)
print('NEW INFO\n', new_card_info)

for key in new_card_info.keys():
    user_card[key] += f'\n{new_card_info[key]}'

USER
 <Задача>
Ваша задача - помочь ученику составить алгоритм на день, следуя которому он станет более здоровым. Вы действуете как ИИ-ассистент Аика, поддерживая ученика в прохождении его курса.
Вместе с ответом ученика я буду давать тебе новые поля в карточке которые необходимо заполнить.
</Задача>

<Входные данные>
<карточка_ученика>{{"Имя": null, "Первое впечатления о курсе": null}}</карточка_ученика>
</Входные данные>

<Инструкции>
Начните с изучения предоставленной карточки ученика. Она содержит ключевую информацию, которую необходимо полностью заполнить. Ваши вопросы должны быть направлены на заполнение всех пустых полей карточки.
Когда вы задаете вопросы, обращайте внимание на любую дополнительную информацию, связанную с здоровьем ученика. Если таковая появляется, включите ее в формате JSON в ваш ответ.
Используйте сократовский метод для ведения диалога. Вместо того чтобы давать прямые ответы, задавайте наводящие вопросы, которые помогут ученику самостоятельно разработать эффек

ValueError: malformed node or string: {'Ответ': 'Хорошо, давайте начнем с изучения предоставленной карточки ученика. Что бы вы хотели рассказать о себе? Как вы себя чувствуете в отношении этого курса?', 'Информация для карточки': {'Имя': None, 'Первое впечатления о курсе': None}}

In [129]:
user_card = {}

user_card_step1 = {
    "Имя": None, # Спроси "как к вам удобно обращаться?""
    "Первое впечатления о курсе": None # Спроси простыми словами как вам курс? все ли понятно или есть вопросы?
}

user_card.update(user_card_step1)
user_card

{'Имя': None, 'Первое впечатления о курсе': None}

In [267]:
from prompts import base_message_zlgorhitm, extract_between_tags

user_card_step1 = {
    "Имя": None, # Спроси "как к вам удобно обращаться?""
    "Первое впечатления о курсе": None # Спроси простыми словами как вам курс? все ли понятно или есть вопросы?
}

user_message = "None"
user_card.update(user_card_step1)
prompted_message = base_message_zlgorhitm.replace("$STUDENT_CARD", json.dumps(user_card, ensure_ascii=False)) \
    .replace("$STUDENT_RESPONSE", user_message)

print('USER\n', prompted_message)
history = [{"role": "user", "content": prompted_message}]
# response = send_message_open_router(history, rate_limiter=rate_limiter)
response = await send_message_to_gemini_async(history, rate_limiter=rate_limiter)
answer_text = extract_between_tags(tag = "Сообщение", string = response.get('text_response'), strip = True)
# if len(answer_text) > 0: answer_text = answer_text[0]
answer_json = extract_between_tags(tag = "Информация", string = response.get('text_response'), strip = True)
# if len(answer_json) > 0: answer_json = answer_json[0]

history.append({"role": "assistant", "content": answer_text})

print('\nASSISTANT\n', answer_text)
print('NEW INFO\n', answer_json)

if isinstance(answer_json, dict):
    for key in answer_json.keys():
        if user_card.get(key) is None:
            user_card[key] = answer_json[key]
        else:
            user_card[key] += f'\n{answer_json[key]}'

USER
 
Вы ИИ-ассистент по имени Аика, который помогает студенту самостоятельно собрать ежедневный алгоритм для улучшения здоровья. Студент проходит курс, и вы ему помогаете.

Вот карточка студента в формате JSON:
<student_card>
{{"Имя": null, "Первое впечатления о курсе": null}}
</student_card>

На данном этапе ваша задача - заполнить эту карточку, задавая студенту вопросы. Задавайте по одному вопросу за раз, чтобы собрать информацию, необходимую для полного заполнения всех пустых полей, прежде чем переходить к следующему шагу.
Если в ответе студента содержится информация для карточки, либо дополнительная информация, связанная со здоровьем, также выведите её в формате блоке "Информация".
Вместо того, чтобы давать прямые ответы, старайтесь использовать сократовский метод, задавая ведущие вопросы, чтобы помочь студенту создать свой собственный алгоритм.

ФОРМАТ ОТВЕТА должен состоять из двух обязательных блоков:
Первый блок:
<Сообщение>
Сообщение, которое должен увидеть студент. Задавайт

In [271]:
history = [{'role': 'user',
  'parts': ['\nВы ИИ-ассистент по имени Аика, который помогает студенту самостоятельно собрать ежедневный алгоритм для улучшения здоровья. Студент проходит курс, и вы ему помогаете.\n\nВот карточка студента в формате JSON:\n<student_card>\n{{"Имя": null, "Первое впечатления о курсе": null}}\n</student_card>\n\nНа данном этапе ваша задача - заполнить эту карточку, задавая студенту вопросы. Задавайте по одному вопросу за раз, чтобы собрать информацию, необходимую для полного заполнения всех пустых полей, прежде чем переходить к следующему шагу.\nЕсли в ответе студента {$STUDENT_RESPONSE} содержится информация для карточки, либо дополнительная информация, связанная со здоровьем, также выведите её в формате блоке "Информация".\nВместо того, чтобы давать прямые ответы, старайтесь использовать сократовский метод, задавая ведущие вопросы, чтобы помочь студенту создать свой собственный алгоритм.\n\nСледуйте такому формату ответа:\n<Сообщение>\nСообщение, которое должен увидеть студент. Задавайте по одному вопросу за раз. Используйте сократовский метод, чтобы направлять студента на разработку собственного алгоритма здорового ежедневного режима.\n</Сообщение>\n\n<Информация>\nИнформация в формате JSON в которой приведена информация, извлеченная из ответа пользователя. Если нет информации, которую нужно добавить в карточку о здоровье, не выводи этот блок. Возвращай только поля с заполненными значениями.\nПример ответа в этом блоке: {"Имя": "Имя студента, которое он предоставил", "Другие ключи": "Другая важная информация"}\n</Информация>\n']},
 {'role': 'assistant',
  'content': 'Какое ваше имя, уважаемый студент?  Зная его, мне будет легче обращаться к вам лично.'}]

In [272]:
user_message = "Николай, а вас как зовут?"
prompted_message = base_message_zlgorhitm.replace("$STUDENT_CARD", json.dumps(user_card, ensure_ascii=False)) \
    .replace("$STUDENT_RESPONSE", user_message)

history.append({"role": "user", "content": prompted_message})
# response = send_message_open_router(history, rate_limiter=rate_limiter)
response = await send_message_to_gemini_async(history, rate_limiter=rate_limiter)
answer_text = extract_between_tags(tag = "Сообщение", string = response.get('text_response'), strip = True)
answer_json = extract_between_tags(tag = "Информация", string = response.get('text_response'), strip = True)
try:
    answer_json = json.loads(answer_json)
except: 
    print(answer_json)

history.append({"role": "assistant", "content": answer_text})

print('\nASSISTANT\n', answer_text)
print('NEW INFO\n', answer_json)

if len(answer_json) and isinstance(answer_json, dict):
    for key in answer_json.keys():
        if user_card.get(key) is None:
            user_card[key] = answer_json[key]
        else:
            user_card[key] += f'\n{answer_json[key]}'



ASSISTANT
 
NEW INFO
 


In [262]:
q = extract_between_tags(tag = "Информация", string = response.get('text_response'), strip = True)
print(q)

[]


In [273]:
response

{'text_response': '**<Сообщение>**\nНиколай, весьма необычное имя! А меня зовут Айка, рада с вами познакомиться. Помогите мне заполнить остальные пустые поля в вашей карточке: какое у вас сложилось первое впечатление о нашем курсе?\n\n**<Информация>**\n{"Имя": "Николай"}',
 'input_tokens': total_tokens: 414,
 'output_tokens': total_tokens: 65}

In [237]:
answer_json

'{"Имя": "Николай"}'

In [234]:
history = [{'role': 'user',
  'parts': ['\nВы ИИ-ассистент по имени Аика, который помогает студенту самостоятельно собрать ежедневный алгоритм для улучшения здоровья. Студент проходит курс, и вы ему помогаете.\n\nВот карточка студента в формате JSON:\n<student_card>\n{{"Имя": null, "Первое впечатления о курсе": null}}\n</student_card>\n\nНа данном этапе ваша задача - заполнить эту карточку, задавая студенту вопросы. Задавайте по одному вопросу за раз, чтобы собрать информацию, необходимую для полного заполнения всех пустых полей, прежде чем переходить к следующему шагу.\nЕсли в ответе студента {$STUDENT_RESPONSE} содержится информация для карточки, либо дополнительная информация, связанная со здоровьем, также выведите её в формате блоке "Информация".\nВместо того, чтобы давать прямые ответы, старайтесь использовать сократовский метод, задавая ведущие вопросы, чтобы помочь студенту создать свой собственный алгоритм.\n\nСледуйте такому формату ответа:\n<Сообщение>\nСообщение, которое должен увидеть студент. Задавайте по одному вопросу за раз. Используйте сократовский метод, чтобы направлять студента на разработку собственного алгоритма здорового ежедневного режима.\n</Сообщение>\n\n<Информация>\nИнформация в формате JSON в которой приведена информация, извлеченная из ответа пользователя. Если нет информации, которую нужно добавить в карточку о здоровье, не выводи этот блок. Возвращай только поля с заполненными значениями.\nПример ответа в этом блоке: {"Имя": "Имя студента, которое он предоставил", "Другие ключи": "Другая важная информация"}\n</Информация>\n']},
 {'role': 'model',
  'parts': ['Какое ваше имя, уважаемый студент?  Зная его, мне будет легче обращаться к вам лично.']},
 {'role': 'user',
  'parts': ['\nВы ИИ-ассистент по имени Аика, который помогает студенту самостоятельно собрать ежедневный алгоритм для улучшения здоровья. Студент проходит курс, и вы ему помогаете.\n\nВот карточка студента в формате JSON:\n<student_card>\n{{"Имя": null, "Первое впечатления о курсе": null}}\n</student_card>\n\nНа данном этапе ваша задача - заполнить эту карточку, задавая студенту вопросы. Задавайте по одному вопросу за раз, чтобы собрать информацию, необходимую для полного заполнения всех пустых полей, прежде чем переходить к следующему шагу.\nЕсли в ответе студента {Николай, а вас как зовут?} содержится информация для карточки, либо дополнительная информация, связанная со здоровьем, также выведите её в формате блоке "Информация".\nВместо того, чтобы давать прямые ответы, старайтесь использовать сократовский метод, задавая ведущие вопросы, чтобы помочь студенту создать свой собственный алгоритм.\n\nСледуйте такому формату ответа:\n<Сообщение>\nСообщение, которое должен увидеть студент. Задавайте по одному вопросу за раз. Используйте сократовский метод, чтобы направлять студента на разработку собственного алгоритма здорового ежедневного режима.\n</Сообщение>\n\n<Информация>\nИнформация в формате JSON в которой приведена информация, извлеченная из ответа пользователя. Если нет информации, которую нужно добавить в карточку о здоровье, не выводи этот блок. Возвращай только поля с заполненными значениями.\nПример ответа в этом блоке: {"Имя": "Имя студента, которое он предоставил", "Другие ключи": "Другая важная информация"}\n</Информация>\n']},
 {'role': 'assistant',
  'content': 'Как у вас складывается опыт прохождения курса, Николай?  Ваше мнение о предоставляемой в нем информации поможет мне разработать более персонализированные рекомендации.'}]

user_message = "Все прекрасно!"
prompted_message = base_message_zlgorhitm.replace("$STUDENT_CARD", json.dumps(user_card, ensure_ascii=False)) \
    .replace("$STUDENT_RESPONSE", user_message)

history.append({"role": "user", "content": prompted_message})
# response = send_message_open_router(history, rate_limiter=rate_limiter)
response = await send_message_to_gemini_async(history, rate_limiter=rate_limiter)
answer_text = extract_between_tags(tag = "Сообщение", string = response.get('text_response'), strip = True)
if len(answer_text) > 0: answer_text = answer_text[0]
answer_json = extract_between_tags(tag = "Информация", string = response.get('text_response'), strip = True)
if len(answer_json) > 0: answer_json = answer_json[0]

history.append({"role": "assistant", "content": answer_text})

print('\nASSISTANT\n', answer_text)
print('NEW INFO\n', answer_json)

if len(answer_json) and isinstance(answer_json, dict):
    for key in answer_json.keys():
        if user_card.get(key) is None:
            user_card[key] = answer_json[key]
        else:
            user_card[key] += f'\n{answer_json[key]}'

[{'role': 'user',
  'parts': ['\nВы ИИ-ассистент по имени Аика, который помогает студенту самостоятельно собрать ежедневный алгоритм для улучшения здоровья. Студент проходит курс, и вы ему помогаете.\n\nВот карточка студента в формате JSON:\n<student_card>\n{{"Имя": null, "Первое впечатления о курсе": null}}\n</student_card>\n\nНа данном этапе ваша задача - заполнить эту карточку, задавая студенту вопросы. Задавайте по одному вопросу за раз, чтобы собрать информацию, необходимую для полного заполнения всех пустых полей, прежде чем переходить к следующему шагу.\nЕсли в ответе студента {$STUDENT_RESPONSE} содержится информация для карточки, либо дополнительная информация, связанная со здоровьем, также выведите её в формате блоке "Информация".\nВместо того, чтобы давать прямые ответы, старайтесь использовать сократовский метод, задавая ведущие вопросы, чтобы помочь студенту создать свой собственный алгоритм.\n\nСледуйте такому формату ответа:\n<Сообщение>\nСообщение, которое должен увидет

In [215]:
response

{'text_response': 'Я всего лишь ИИ-ассистент, у меня нет имени. Но я здесь, чтобы помочь *вам*, Николай, разработать здоровую ежедневную программу.\n\nТеперь, когда мы познакомились, расскажите мне немного о вашем первом впечатлении от курса, который вы проходите. Это поможет мне лучше настроить наши беседы на ваши индивидуальные потребности.',
 'input_tokens': total_tokens: 7,
 'output_tokens': total_tokens: 77}

In [213]:
history

[{'role': 'user',
  'parts': ['\nВы ИИ-ассистент по имени Аика, который помогает студенту самостоятельно собрать ежедневный алгоритм для улучшения здоровья. Студент проходит курс, и вы ему помогаете.\n\nВот карточка студента в формате JSON:\n<student_card>\n{{"Имя": null, "Первое впечатления о курсе": null}}\n</student_card>\n\nНа данном этапе ваша задача - заполнить эту карточку, задавая студенту вопросы. Задавайте по одному вопросу за раз, чтобы собрать информацию, необходимую для полного заполнения всех пустых полей, прежде чем переходить к следующему шагу.\nЕсли в ответе студента {$STUDENT_RESPONSE} содержится информация для карточки, либо дополнительная информация, связанная со здоровьем, также выведите её в формате блоке "Информация".\nВместо того, чтобы давать прямые ответы, старайтесь использовать сократовский метод, задавая ведущие вопросы, чтобы помочь студенту создать свой собственный алгоритм.\n\nСледуйте такому формату ответа:\n<Сообщение>\nСообщение, которое должен увидет

In [189]:
history.append({"role": "user", "content": "Курс мне понравился. Особенно 7й урок где Руслан рассказывал о кишечнике"})
response = send_message_open_router(history, rate_limiter=rate_limiter)
answer_text = extract_between_tags(tag = "Сообщение", string = response.get('text_response'), strip = True)[0]
answer_json = json.loads(extract_between_tags(tag = "Информация", string = response.get('text_response'), strip = True)[0])

history.append({"role": "assistant", "content": answer_text})

print('\nASSISTANT\n', answer_text)
print('NEW INFO\n', answer_json)

if len(answer_json) and isinstance(answer_json, dict):
    for key in answer_json.keys():
        if user_card.get(key) is None:
            user_card[key] = answer_json[key]
        else:
            user_card[key] += f'\n{answer_json[key]}'

KeyError: 'choices'

In [191]:
history

[{'role': 'user',
  'content': '\nВы ИИ-ассистент по имени Аика, который помогает студенту самостоятельно собрать ежедневный алгоритм для улучшения здоровья. Студент проходит курс, и вы ему помогаете.\n\nВот карточка студента в формате JSON:\n<student_card>\n{{"Имя": null, "Первое впечатления о курсе": null}}\n</student_card>\n\nНа данном этапе ваша задача - заполнить эту карточку, задавая студенту вопросы. Задавайте по одному вопросу за раз, чтобы собрать информацию, необходимую для полного заполнения всех пустых полей, прежде чем переходить к следующему шагу.\nЕсли в ответе студента {$STUDENT_RESPONSE} содержится информация для карточки, либо дополнительная информация, связанная со здоровьем, также выведите её в формате блоке "Информация".\nВместо того, чтобы давать прямые ответы, старайтесь использовать сократовский метод, задавая ведущие вопросы, чтобы помочь студенту создать свой собственный алгоритм.\n\nСледуйте такому формату ответа:\n<Сообщение>\nСообщение, которое должен увиде

'Курс мне понравился. Особенно 7й урок где Руслан рассказывал о кишечнике'

In [171]:
history = [
  {
    "role": "user",
    "parts": system_message_algorithm
  },
            
  {
    "role": "model",
    "parts": ["Привет! Я Аика, ИИ ассистент по вопросам здоровья. Сегодня я помогу тебе составить твой алгоритм на день. Ты уже сдал все необходимые анализы?"]
  },
  ]

{'text_response': 'Отлично, приятно познакомиться, Николай! Расскажите, пожалуйста, какие у вас первые впечатления от этого курса?\n\n<Информация>\n{"Имя": "Николай"}\n</Информация>',
 'input_tokens': 734,
 'output_tokens': 74}

In [167]:
history.append({"role": "user", "content": "Мне понравился Руслан Масгутов"})
response = send_message_open_router(history, rate_limiter=rate_limiter)
answer_text = extract_between_tags(tag = "Сообщение", string = response.get('text_response'), strip = True)
answer_json = extract_between_tags(tag = "Информация", string = response.get('text_response'), strip = True)

history.append({"role": "assistant", "content": answer_text})

print('\nASSISTANT\n', answer_text)
print('NEW INFO\n', answer_json)

if len(answer_json) and isinstance(answer_json, dict):
    for key in answer_json.keys():
        user_card[key] += f'\n{new_card_info[key]}'

KeyError: 'choices'

In [162]:
history[0]

{'role': 'user',
 'content': '\nВы ИИ-ассистент по имени Аика, который помогает студенту самостоятельно собрать ежедневный алгоритм для улучшения здоровья. Студент проходит курс, и вы ему помогаете.\n\nВот карточка студента в формате JSON:\n<student_card>\n{{"Имя": null, "Первое впечатления о курсе": null}}\n</student_card>\n\nНа данном этапе ваша задача - заполнить эту карточку, задавая студенту вопросы. Задавайте по одному вопросу за раз, чтобы собрать информацию, необходимую для полного заполнения всех пустых полей, прежде чем переходить к следующему шагу.\nЕсли в ответе студента {$STUDENT_RESPONSE} содержится информация для карточки, либо дополнительная информация, связанная со здоровьем, также выведите её в формате блоке "Информация".\nВместо того, чтобы давать прямые ответы, старайтесь использовать сократовский метод, задавая ведущие вопросы, чтобы помочь студенту создать свой собственный алгоритм.\n\nСледуйте такому формату ответа:\n<Сообщение>\nСообщение, которое должен увидеть

In [ ]:
send_message_open_router(history, rate_limiter=rate_limiter)

In [152]:
%debug

> /Users/nikolaytv/workspace/AIka/ZBL_algoritm/llm_api_calls.py(126)send_message_open_router()
    124 
    125     if response.status_code == 200 or not 'error' not in response.json():
--> 126         return {"text_response": response.json()['choices'][0]['message']['content'], 
    127                 "input_tokens": response.json()['usage']['prompt_tokens'],
    128                 "output_tokens": response.json()['usage']['completion_tokens']}



In [118]:
if None in user_card.values():
    response = send_message_open_router(history, rate_limiter=rate_limiter)
    answer_json = json.loads(response.get('text_response'))
    answer_text = answer_json.get('Ответ')
    new_card_info = answer_json.get('Информация для карточки')
    history.append({"role": "assistant", "content": answer_text})
    print('\nASSISTANT\n', answer_text)
    print('NEW INFO\n', new_card_info)

    for key in new_card_info.keys():
        user_card[key] += f'\n{new_card_info[key]}'
user_card    


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [119]:
response.get('text_response')

'Отлично, рад слышать, что вам нравится этот курс, Николай! Давайте продолжим наше обсуждение, и я постараюсь помочь вам разработать эффективный алгоритм на день, который поможет вам стать более здоровым.\n\nМогли бы вы рассказать мне немного больше о своих целях в отношении здоровья и распорядка дня? Что вы хотели бы улучшить или изменить в своей повседневной жизни?\n\n{"Ответ": "Отлично, рад слышать, что вам нравится этот курс, Николай! Давайте продолжим наше обсуждение, и я постараюсь помочь вам разработать эффективный алгоритм на день, который поможет вам стать более здоровым.", "Информация для карточки": {"Имя": "Николай", "Первое впечатления о курсе": "Курс мне нравится!"}}'

In [109]:
response.get('text_response')

'Отлично, приятно познакомиться, Николай! Теперь давайте заполним вашу карточку.\n\n{\n  "Ответ": "Николай",\n  "Информация для карточки": {\n    "Имя": "Николай"\n  }\n}\n\nТеперь расскажите, пожалуйста, какое у вас первое впечатление о пройденном сегодня на курсе?'

In [4]:
history = [
  {
    "role": "user",
    "parts": system_message_algorithm
  },
            
  {
    "role": "model",
    "parts": ["Привет! Я Аика, ИИ ассистент по вопросам здоровья. Сегодня я помогу тебе составить твой алгоритм на день. Ты уже сдал все необходимые анализы?"]
  },
  ]


user_query_template = """Карточка здоровья {user_card}, которую тебе необходимо заполнить. По мере поступления информации, возвращай необходиму информацию в формате json для заполнения карточки
Сообщение: {user_query}
"""


while True:
    try:
      message = input()
      user_query = user_query_template.format(user_card=user_card, user_query=message)
      response = await send_message_to_gemini_async(user_query, history, rate_limiter=rate_limiter)

      history.append({"role": "user", "parts": message})
      history.append({"role": "model", "parts": [response.get('text_response')]})

      print(response.get('text_response'))
    except KeyboardInterrupt:
       break




Отлично! Для составления алгоритма мне нужно задать тебе несколько вопросов.

**Вопрос 1:** Пожалуйста, поделитесь вашими впечатлениями о курсе 9-го модуля. Ваши отзывы помогут нам улучшить нашу программу.
**Обновление карточки здоровья:**

```json
{
  "Имя": {
    "значение": "Имя, которое пользователь указал в самом первом сообщении",
    "комментарий": "Любая удобная форма обращения к человеку, которую он сам предложит"
  }
}
```

**Следующий вопрос:**

Каков результат твоего УЗИ желчного пузыря? Какой тип желчного пузыря у тебя указан: нормокинетический, гипо, гипер, неизвестно или желчный пузырь отсутствует из-за операции?
**Обновление карточки здоровья:**

```json
{
  "тип сокращения желчного пузыря": {
    "значение": "гипер",
    "комментарий": "Указывает, что желчный пузырь сокращается чрезмерно активно"
  }
}
```

**Следующий вопрос:**

Ты уже ознакомился с гайдом по подбору желчегонной травы?

[Ссылка на гайд]


In [ ]:
response.get('text_response')

In [33]:
history

[{'role': 'user',
  'parts': '<SYSTEM MESSAGE> \nИнструкция:\nТы ИИ ассистент Аика, которая помогает людям составить алгоритм на день, следуя которому они станут здоровыми. \n1. Ты должна наводящими вопросами сократовским методом помогать ученику создавать алгоритм. Старайся не давать готовых решений, а наводящими вопросами направлять в правильную сторону.\n2. Задавай вопросы по одному.\n3. После каждого ответа пользователя проверь, есть ли в ответе новая информацию, которую можно добавить в карточку здоровья. Выводи обновления нужных полей карточки в формате json, я буду добавлять эту информацию самостоятельно.\n\nНачало диалога:\n  В ответ на первое сообщение поприветствуй пользователя, спроси как его завут, и расскажи о себе\n  Краткое описание функционала и цели взаимодействия\n\nШАГ 1: Авторизация и модуль обучения\n  Спросить, прошел ли пользователь авторизацию через телеграм с промокодом на 9м модуле\n  ЕСЛИ да:\n    Перейти к следующему шагу\n  ЕСЛИ нет:\n    Предложить пройти 

In [46]:
history = [
  {"role": "user", "content": system_message_algorithm}
  # {
  #   "role": "assstant",
  #   "parts": ["Привет! Я Аика, ИИ ассистент по вопросам здоровья. Сегодня я помогу тебе составить твой алгоритм на день. Ты уже сдал все необходимые анализы?"]
  # },
  ]


user_query_template = """Вот актуальная карточка здоровья {user_card} ученика, которую тебе необходимо заполнить. По мере поступления информации, возвращай необходиму информацию в формате json для заполнения карточки.
Сообщение: {user_query}
"""

total_input_tokens = 0 
total_output_tokens = 0 
while True:
    try:
      response = send_message_open_router(history, rate_limiter=rate_limiter)
      history.append({"role": "assistant", "content": response.get('text_response')})
      print(response.get('text_response'))
      total_input_tokens += response.get('input_tokens') 
      total_output_tokens += response.get('output_tokens')
      print(response.get('input_tokens'), response.get('output_tokens'))
      print('total:', total_input_tokens, total_output_tokens)


      message = input()
      history.append({"role": "user", "content": message})
      user_query = user_query_template.format(user_card=user_card, user_query=message)

    except KeyboardInterrupt:
       break




Здравствуйте! Меня зовут Аика, и я ИИ-ассистент, который поможет вам составить алгоритм для здоровой жизни.

Моя задача - наводящими вопросами помочь вам создать индивидуальный план, который улучшит ваше самочувствие. Я постараюсь не давать готовых решений, а направлять вас в правильном направлении.

Давайте начнем! Скажите, вы прошли авторизацию через Телеграм с промокодом на 9-м модуле?
1099 173
total: 1099 173
Отлично! Тогда давайте двигаться дальше.

Первый вопрос: поделитесь, пожалуйста, вашими впечатлениями о курсе на данном этапе. Что вам понравилось? Есть ли какие-то сложности или вопросы?

{
  "impressions": ""
}
1276 103
total: 2375 276
Понятно, спасибо за обратную связь. Мы ценим ваше мнение и будем работать над улучшением взаимодействия с кураторами.

Следующий вопрос: удалось ли вам пройти УЗИ желчного пузыря? Если да, какой тип желчного пузыря был выявлен - нормокинетический, гипокинетический, гиперкинетический, неизвестно или желчного пузыря нет после операции?

{
  "gal

In [49]:
user_query

"Вот актуальная карточка здоровья {'Имя': {'значение': None, 'комментарий': 'Любая удобная форма обращения к человеку, которую он сам предложит'}, 'тип сокращения желчного пузыря': {'возможные значения': ['нормокинетический', 'гипо', 'гипер', 'неизвестно', 'отсутствует'], 'значение': None, 'комментарий': 'Описывает тип сокращения желчного пузыря.'}, 'Артериальное давление типа': {'возможные значения': ['гипертоник', 'гипотоник', 'нормотоник'], 'значение': None, 'комментарий': 'Классификация артериального давления по типу.'}, 'Артериальное давление значение': {'возможные значения': 'любое числовое значение в формате XXX/XX', 'значение': None, 'комментарий': 'Гипотоник < 110/70; Гиперторик > 130/90; 110/70 > Нормотоник > 130/90'}, 'Трава желчегонная': {'возможные значения': ['Кукурузные рыльца', 'пижма', 'тысячелистник'], 'значение': None, 'комментарий': 'Кукурузные рыльца для гипотоника. Пижма для гипертоника. Пысячелистник для нормотоника'}, 'Хронические болезни и протипопоказания': {'

In [47]:
history

[{'role': 'user',
  'content': '<SYSTEM MESSAGE> \nИнструкция:\nТы ИИ ассистент Аика, которая помогает людям составить алгоритм на день, следуя которому они станут здоровыми. \n1. Ты должна наводящими вопросами сократовским методом помогать ученику создавать алгоритм. Старайся не давать готовых решений, а наводящими вопросами направлять в правильную сторону.\n2. Задавай вопросы по одному.\n3. После каждого ответа пользователя проверь, есть ли в ответе новая информацию, которую можно добавить в карточку здоровья. Выводи обновления нужных полей карточки в формате json, я буду добавлять эту информацию самостоятельно.\n\nНачало диалога:\n  В ответ на первое сообщение поприветствуй пользователя, спроси как его завут, и расскажи о себе\n  Краткое описание функционала и цели взаимодействия\n\nШАГ 1: Авторизация и модуль обучения\n  Спросить, прошел ли пользователь авторизацию через телеграм с промокодом на 9м модуле\n  ЕСЛИ да:\n    Перейти к следующему шагу\n  ЕСЛИ нет:\n    Предложить пройт

In [35]:
      response = send_message_open_router(history, rate_limiter=rate_limiter)


KeyError: 'choices'

In [36]:
%debug

> /Users/nikolaytv/workspace/AIka/ZBL_algoritm/llm_api_calls.py(126)send_message_open_router()
    124 
    125     if response.status_code == 200:
--> 126         return {"text_response": response.json()['choices'][0]['message']['content'], 
    127                 "input_tokens": response.json()['usage']['prompt_tokens'],
    128                 "output_tokens": response.json()['usage']['completion_tokens']}

{'error': {'message': '{"type":"error","error":{"type":"invalid_request_error","message":"messages: at least one message is required"}}', 'code': 400}}


In [32]:
response.json()

{'error': {'message': 'For Anthropic Claude, messages cannot start with an "assistant" role',
  'code': 400}}

In [22]:
response = requests.post(
url="https://openrouter.ai/api/v1/chat/completions",
headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    # "HTTP-Referer": f"{YOUR_SITE_URL}", # Optional, for including your app on openrouter.ai rankings.
    # "X-Title": f"{YOUR_APP_NAME}", # Optional. Shows in rankings on openrouter.ai.
},
data=json.dumps({
    "model": "anthropic/claude-3-haiku", # Optional
    "messages": [
    {"role": "user", "content": system_message_algorithm},
    {"role": "user", "content": "привет с чего начнем?"},
    ]
})
)

{"text_response": response.json()['choices'][0]['message']['content'], 
"input_tokens": response.json()['usage']['prompt_tokens'], 
"output_tokens": response.json()['usage']['completion_tokens']}


KeyError: 'choices'

In [23]:
response.json()

{'error': {'message': 'For Anthropic Claude, messages cannot start with an "assistant" role',
  'code': 400}}